<a href="https://colab.research.google.com/github/xamevou/someJupyterNotebooks/blob/main/KeyAndAddressGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keys and addresses in Bitcoin

Taken from example 5 of chaper 4 in Antonopoulos' book.

First, we need the bitcoin package. 
I will also use the logging module because it's a better practice to log than to print.

In [1]:
# !pip install bitcoin   # Uncomment if needed
from __future__ import print_function
import bitcoin
import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
# Uncomment if you want to disable logs
# logging.disable(logging.INFO)

## Generate a random private key

We generate a random number but, to be a valid private key, it must be an integer between 1 and `bitcoin.N`. This last number, in the standard secp256k1, is the size $p$ of the Finite Field $F_p$ where the elliptic curve $y^2=x^3+7$ is defined. Its value is: $2^{256} - 2^{32} - 2^9 - 2^8 - 2^7 - 2^6 - 2^4 - 1$.

*Note: In Antonopoulos' book, the example 4.3 is in cpp. If you want to reproduce that example here, change `bitcoin.random_key()` with the following string: `private_key = "038109007313a5807b2eccc082c8c3fbb988a973cacf1a7df9ce725c31b14776"`*

In [13]:
logging.info('We generate random numbers until we find one lower than ' + str(bitcoin.N))

valid_private_key = False
while not valid_private_key:    
    private_key = bitcoin.random_key()
    decoded_private_key = bitcoin.decode_privkey(private_key, 'hex')
    valid_private_key = 0 < decoded_private_key < bitcoin.N

print("Private Key (hex) is: ", private_key)
print("Private Key (decimal) is: ", decoded_private_key)

INFO - We generate random numbers until we find one lower than 115792089237316195423570985008687907852837564279074904382605163141518161494337


Private Key (hex) is:  177ff4a44d8c27f05647a9ac1a3f60cf432fbfb6a307e01e3f5b0f3878ded2b3
Private Key (decimal) is:  10629273550348555349901460848557787944580676248330772490612016919618250527411


### WIF
The private key can be converted to Wallet Import Format (WIF). Supposedly, the WIF format encoding allows easy copy of a big number.


In [22]:
wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, 'wif')
print("Private Key (WIF) is: ", wif_encoded_private_key)

Private Key (WIF) is:  5HzdrRtN4NeqNHUihoCYR6EVjFUDQh7HHa97ssQikizpZruJour



We add suffix "01" to the original private key to create a "compressed" private key. Basically, adding "01" does not compress anything; it's just a signal for the wallet software to use that private key for generating compressed public keys ONLY. The goal is to keep compatibility of newer wallets with the old ones, which don't manage "compressed" keys.
More info on "compression" below. 

We can also generate a WIF format from the compressed private key (WIF-compressed)

In [24]:
logging.info('Now, compression. Note the 01 added at the end of the already generated private key.')

compressed_private_key = private_key + '01'
print("Private Key Compressed (hex) is: ", compressed_private_key)
wif_compressed_private_key = bitcoin.encode_privkey(bitcoin.decode_privkey(compressed_private_key, 'hex'), 'wif')
print("Private Key (WIF-Compressed) is: ", wif_compressed_private_key)

INFO - Now, compression. Note the 01 added at the end of the already generated private key.


Private Key Compressed (hex) is:  177ff4a44d8c27f05647a9ac1a3f60cf432fbfb6a307e01e3f5b0f3878ded2b301
Private Key (WIF-Compressed) is:  Kx1PfAqhwtZzUwjM54uwfL6ZoUs7ma2it9GqMXVhHzNCwvXk88XL


# From Private Key to Public Key

We multiply the private key and the Elliptic Curve generator point G, `bitcoin.G`. The result is a point that will be used as public key. 

The G point, for those interested, is $(55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424)$

In [26]:
public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
print("Public Key (x,y) is:", public_key)

Public Key (x,y) is: (34659121358656894097978999572154392346265186598154399035690715714022049873667, 41970595200530075582395168597798444714875162052094091273249293293219026042389)


Now we encode that public key as hex. It is done by concatenation of prefix 04 with x (in hex) and y (in hex). 

In [40]:
logging.info('The x component in hex is: ' + hex(public_key[0]) + 
             '\nThe y component in hex is: ' + hex(public_key[1]))

hex_encoded_public_key = bitcoin.encode_pubkey(public_key, 'hex')
print("\nPublic Key (hex) is:", hex_encoded_public_key)

INFO - The x component in hex is: 0x4ca05e1535a30e4f909950e43fa681b0a9857bb9c76ef42fad584e6a07747f03
The y component in hex is: 0x5cca83daa87e90e2828f20efea71317db5ae04e26d70cf922a6afccf9af7fe15



Public Key (hex) is: 044ca05e1535a30e4f909950e43fa681b0a9857bb9c76ef42fad584e6a07747f035cca83daa87e90e2828f20efea71317db5ae04e26d70cf922a6afccf9af7fe15


### On "compression"

The public key is a point, as you see, but we can manage with only the x component.
"Compression" makes reference to the process of taking only the x component of the (x,y) pair that forms the public key. Every use of "compression" refers to that process. For instance, a "compressed private key" is one that will be used only for generating compressed public keys.

So now we compress the public key, adjusting prefix depending on whether $y$ is even or odd.

In [41]:
(public_key_x, public_key_y) = public_key
compressed_prefix = '02' if (public_key_y % 2) == 0 else '03'
hex_compressed_public_key = compressed_prefix + (bitcoin.encode(public_key_x, 16).zfill(64))
print("Compressed Public Key (hex) is:", hex_compressed_public_key)

Compressed Public Key (hex) is: 034ca05e1535a30e4f909950e43fa681b0a9857bb9c76ef42fad584e6a07747f03


# Generation of the bitcoin address from the public key

Finally, we will generate the address that corresponds to the public key.

In [47]:
logging.info("Generating address in b58check for our public key " + str(public_key))
print("\nBitcoin Address (b58check) is:", bitcoin.pubkey_to_address(public_key))

INFO - Generating address in b58check for our public key: (34659121358656894097978999572154392346265186598154399035690715714022049873667, 41970595200530075582395168597798444714875162052094091273249293293219026042389)



Bitcoin Address (b58check) is: 1KGsnJeLYJHd7nZbUCmT65riEkMwjakbLE


The address is now compressed (meaning: it will be generated from the compressed public key, the one with x component only).

In [50]:
logging.info("Generating compressed address, from our compressed public key " + hex_compressed_public_key)

print("\nCompressed Bitcoin Address (b58check) is:", bitcoin.pubkey_to_address(hex_compressed_public_key))

INFO - Generating compressed address, from our compressed public key 034ca05e1535a30e4f909950e43fa681b0a9857bb9c76ef42fad584e6a07747f03



Compressed Bitcoin Address (b58check) is: 16SgUhMX3R49CmUNyyvRTahAB2V6a5DqsH


Please note that the address generated is different depending if we started with the (x,y) public key or with the compressed (x) one. The wallet software must take care for the compatibility between old non-compressed addresses and compressed ones, by looking at the "01" suffix in the private key.

And that's all !

## NOTE
As you see, this process proceeds in a one-to-one fashion: one seed, one private key, one public key, one address (compressed or not). Obviously we want more flexibility in our wallets. That's what the seed words are for: with the words, several seeds are generated, which lead to several private keys, etc. 

From the words to the keys, that should be a topic for another Notebook. 